## Documents

In [1]:
from glob import glob

with open('docs.txt') as fp:
    docs = [d.strip() for d in fp.readlines()]
docs_r = {k:i for i, k in enumerate(docs)}

In [2]:
docs[:20]

['../dataset/IR_dataset/2048.txt',
 '../dataset/IR_dataset/2404.txt',
 '../dataset/IR_dataset/661.txt',
 '../dataset/IR_dataset/1252.txt',
 '../dataset/IR_dataset/726.txt',
 '../dataset/IR_dataset/3029.txt',
 '../dataset/IR_dataset/329.txt',
 '../dataset/IR_dataset/1481.txt',
 '../dataset/IR_dataset/1511.txt',
 '../dataset/IR_dataset/1127.txt',
 '../dataset/IR_dataset/2447.txt',
 '../dataset/IR_dataset/2907.txt',
 '../dataset/IR_dataset/630.txt',
 '../dataset/IR_dataset/3008.txt',
 '../dataset/IR_dataset/494.txt',
 '../dataset/IR_dataset/321.txt',
 '../dataset/IR_dataset/40.txt',
 '../dataset/IR_dataset/3131.txt',
 '../dataset/IR_dataset/2192.txt',
 '../dataset/IR_dataset/2167.txt']

## Loading Dev Dataset

In [3]:
import yaml

In [4]:
with open('../dataset/evaluation_IR.yml', 'r') as f:
    dataset = yaml.safe_load(f)

In [5]:
len(dataset)

150

In [6]:
dataset_dev = {k:v for k, v in list(dataset.items())[-30:]}

In [7]:
list(dataset_dev.keys())

['نظریه اصل موضوعی مجموعه\u200cها  سازگاری و عدم وابستگی در ZFC',
 'نظریه نسبیت  نسبیت عام',
 'نلسون ماندلا  آغاز',
 'نهنگ قاتل  هوش',
 'نوا (دستگاه موسیقی)  گوشه\u200cها',
 'نیروهای محور  اتحاد دانوب، اختلاف بر سر اتریش',
 'هاینریش هیملر  استحکام قدرت',
 'هاینریش هیملر  رابطه با هیتلر',
 'هسته لینوکس',
 'هسته لینوکس  درگیری\u200cهای جامعه توسعه',
 'هسته لینوکس  مدل توسعه',
 'هم\u200cارزی جرم و انرژی  کاربست\u200cپذیری فرمول',
 'هندسه جبری',
 'هوش مصنوعی  تاریخچه',
 'واپاشی هسته\u200cای  پایداری و ناپایداری ایزوتوپ\u200cها',
 'وشمگیر  وضعیت سیاسی-اجتماعی قرن چهارم هجری',
 'ولایت قندهار  تمدن مندیگک',
 'ولفگانگ آمادئوس موتسارت  موتسارت در وین',
 'ونکوور  سیستم حمل و نقل شهری',
 'ونکوور  معماری',
 'پروین اعتصامی',
 'پرچم ایران  پیش از پادشاهی پهلوی\u200cها',
 'پیمان کیوتو  اتحادیه اروپا',
 'چرخه آب  توصیف',
 'چنگیز خان  کودکی',
 'چهاردهمین دالایی لاما  اوان زندگی و سابقه',
 'کارل مارکس  اقتصاد، تاریخ و جامعه',
 'گرجستان  تاریخ',
 'یانی  تأثیرپذیری\u200cهای موسیقایی',
 'یونسکو  فعالیت\u20

## POS Tagger

In [10]:
import tensorflow
from tensorflow.keras.models import load_model

In [11]:
with open('pos/words.txt') as fp:
    words_all = [line.strip() for line in fp.readlines()]
word2int = {k:i for i, k in enumerate(words_all)}
VOCAB_SIZE = len(word2int) + 1

In [12]:
with open('pos/tags.txt') as fp:
    tags_all = [line.strip() for line in fp.readlines()]
tag2int = {k:i for i, k in enumerate(tags_all)}
TAGS_NO = len(tag2int)

In [13]:
from tensorflow.keras.models import load_model
pos_tagger = load_model('models/pos_lstm_3.h5')

In [14]:
pos_tagger.summary()

In [13]:
SEQ_LEN = 50

In [14]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [15]:
allowed_chars = [
    'آ', 'أ', 'ؤ', 'إ', 'ئ', 'ا', 'ب', 'ة', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش',
    'ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ى','ي','٠','١','٢','٣', '٤', '٥', '٦', '٧',
    '٨', '٩', 'چ', 'ژ', 'ک', 'گ', 'ھ', 'ی', '۰', '۱', '۲', '۳', '۴', '۵', '۶', '۷', '۸', '۹',
    '\u200c', '\u200d', '\u200e', '\u200f',
    'ﭼ', 'ﯽ', 'ﯾ', 'ﯿ', 'ﷲ', 'ﺀ', 'ﺄ', 'ﺆ', 'ﺋ', 'ﺎ', 'ﺑ', 'ﺔ', 'ﺗ', 'ﺘ', 'ﺧ', 'ﺪ', 'ﺮ', 'ﺳ', 'ﺴ', 'ﺿ',
    'ﻋ','ﻌ', 'ﻗ', 'ﻠ', 'ﻣ', 'ﻨ', 'ﻼ', '￼', 'پ',]

trans_chars = [
    'ً', 'ٌ', 'ٍ', 'َ', 'ُ', 'ِ', 'ّ', 'ْ', 'ٓ', 'ٔ',
]

In [16]:
import re
import nltk

def normalize_text(text):
    text = re.sub('[' + ''.join(trans_chars) + ']', '', text)
    text = re.sub('[^' + ''.join(allowed_chars) + ']', ' ', text)
#     text = re.sub('ئ', 'ی', text)
    text = re.sub('ء', '', text)
    text = re.sub('[\s]+', ' ', text)
    return text

def encode_text(text):
    text = normalize_text(text)
    tokens = nltk.tokenize.word_tokenize(text)
    return tokens, [word2int[word] if word in word2int else word2int['[UNK]'] for word in tokens]

In [17]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def pos_tag(text):
    test_X = []
    test_words = []

    text_words, text_tokens = encode_text(text)
    words_list = list(chunks(text_words, SEQ_LEN))
    tokens_list = list(chunks(text_tokens, SEQ_LEN))
    test_X += tokens_list
    test_words += words_list
    test_X = pad_sequences(test_X, maxlen=SEQ_LEN, padding='post')
    pred_outs = pos_tagger.predict(test_X)
    pred_args = np.argmax(pred_outs, axis=2)
    pred_tags = []
    for i, pred in enumerate(pred_args):
        cur_tags = [tags_all[i] if i in range(len(tags_all)) else 'UNK' for i in pred]
        cur_pairs = list(zip(test_words[i], cur_tags))
        pred_tags += cur_pairs
    return pred_tags

### POS Tag Test

In [19]:
from random import sample

with open('../dataset/IR_dataset/78.txt') as fp:
    text = fp.read()
    text = re.sub('\n', ' ', text)
    print(pos_tag(text))

[('اصل', 'N_SING'), ('موضوع', 'N_SING'), ('اجتماع', 'N_SING'), ('بیان', 'N_SING'), ('می\u200cکند', 'V_PRS'), ('یا', 'CON'), ('به', 'P'), ('بیان', 'N_SING'), ('دیگر', 'ADJ'), ('برای', 'P'), ('هر', 'DET'), ('دسته', 'N_SING'), ('دلخواه', 'ADJ'), ('از', 'P'), ('مجموعه\u200cها', 'N_PL'), ('مجموعه\u200cای', 'N_SING'), ('وجود', 'N_SING'), ('دارد', 'V_PRS'), ('که', 'CON'), ('شامل', 'ADJ'), ('همه', 'PRO'), ('عناصری', 'N_PL'), ('است', 'V_PRS'), ('که', 'CON'), ('حداقل', 'ADV'), ('به', 'P'), ('یکی', 'PRO'), ('از', 'P'), ('مجموعه\u200cهای', 'N_PL'), ('دسته', 'N_SING'), ('مفروض', 'N_SING'), ('متعلق', 'ADJ'), ('باشند', 'V_SUB'), ('به', 'P'), ('بیان', 'N_SING'), ('دیگر', 'ADJ'), ('اگر', 'CON'), ('دسته\u200cای', 'N_SING'), ('از', 'P'), ('مجموعه\u200cها', 'N_PL'), ('باشد', 'V_SUB'), ('مجموعه\u200cای', 'N_SING'), ('چون', 'CON'), ('وجود', 'N_SING'), ('دارد', 'V_PRS'), ('که', 'CON'), ('اگر', 'CON'), ('موجود', 'ADJ'), ('باشد', 'V_SUB'), ('به\u200cطوری\u200cکه', 'N_SING'), ('آنگاه', 'ADV'), ('اما', 'CON'), (

## Word2Vec

In [21]:
import numpy as np

word2vec = np.load("w2v/word2vec2.npy")

In [22]:
with open('w2v/vocab2.txt') as fp:
    vocab = [l.strip() for l in fp.readlines()]
vocab_r = {k:i for i, k in enumerate(vocab)}

In [23]:
word2vec.dtype

dtype('float64')

In [23]:
len(vocab), word2vec.shape

(75011, (75011, 200))

In [24]:
EMBED_DIM = 200

In [25]:
def get_word2vec(word):
    return word2vec[vocab_r[word]] if word in vocab_r else np.zeros((EMBED_DIM,))

## POS Tag Weights

In [78]:
tag_w = {
    'PAD': 0,
    'ADJ': 1,
    'ADJ_CMPR': 1,
    'ADJ_INO': 1,
    'ADJ_SUP': 1,
    'ADJ_VOC': 1,
    'ADV': 0.5,
    'ADV_COMP': 0.5,
    'ADV_I': 0.5,
    'ADV_LOC': 0.5,
    'ADV_NEG': 0.5,
    'ADV_TIME': 0.5,
    'CLITIC': 0,
    'CON': 0,
    'DELM': 0,
    'DET': 0,
    'FW': 2,
    'INT': 0,
    'NUM': 2,
    'N_PL': 12,
    'N_SING': 12,
    'N_VOC': 6,
    'P': 0,
    'PREV': 0,
    'PRO': 0,
    'SYM': 0,
    'UNK': 0,
    'V_AUX': 0,
    'V_IMP': 0,
    'V_PA': 0,
    'V_PP': 0,
    'V_PRS': 0,
    'V_SUB': 0
}

## Tf-Idf Vectorization

In [27]:
from scipy.sparse import load_npz

tfidf = load_npz('tfidf/tfidf.npz')

tfidf_words = []
with open('tfidf/words.txt') as fp:
    tfidf_words = [w.strip() for w in fp.readlines()]
tfidf_words_r = {k:i for i, k in enumerate(tfidf_words)}

In [106]:
tfidf[docs_r['../dataset/IR_dataset/2048.txt'], tfidf_words_r['پودر']]

0.30680919856295225

In [28]:
def get_tfidf(doc, word):
    return tfidf[docs_r[doc], tfidf_words_r[word]] if word in tfidf_words_r else 0

## Document Embedding

In [29]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [79]:
def embed_doc(doc, text):
    pos_tags = pos_tag(text)
    vectors = np.zeros((len(pos_tags), EMBED_DIM), dtype=np.float64)
    weights = np.zeros((len(pos_tags),), dtype=np.float64)
    i = 0
    for word, tag in pos_tags:
        vectors[i] = get_word2vec(word)
        weights[i] = tag_w[tag] + get_tfidf(doc, word) # To be tuned
        i += 1
        
    s = softmax(weights)
    return np.dot(s, vectors)

### Test

In [135]:
sel_doc = docs[0]
with open(sel_doc) as fp:
    text = fp.read()
    text = re.sub('\n', ' ', text)
    v = embed_doc(sel_doc, text)
v.shape

(200,)

## Keeping Together

In [31]:
def cos_sim(a, b):
    na = np.linalg.norm(a)
    nb =np.linalg.norm(b) 
    return np.dot(a, b) / (na * nb) if na > 0 and nb > 0 else 0

In [32]:
def embed_query(query):
    pos_tags = pos_tag(query)
    vectors = np.zeros((len(pos_tags), EMBED_DIM), dtype=np.float64)
    weights = np.zeros((len(pos_tags),), dtype=np.float64)
    i = 0
    for word, tag in pos_tags:
        vectors[i] = get_word2vec(word)
        weights[i] = 1 / len(pos_tags)
        i += 1
    return np.dot(weights, vectors)

In [84]:
query = 'هوش مصنوعی  تاریخچه'

q_vec = embed_query(query)
q_vec.shape

(200,)

In [8]:
for query in dataset_dev:
    print(dataset_dev[query])

{'relevant': [2712], 'similar_high': [2713, 2714, 661], 'similar_low': [2724, 2725, 2726, 2727, 2728, 2729, 2463, 2730, 2731, 2732], 'similar_med': [2715, 2716, 2717, 1215, 2718, 2719, 2720, 2721, 2722, 2723]}
{'relevant': [1618], 'similar_high': [1617, 1619, 1620, 1621], 'similar_low': [2562, 1056, 2563, 2564, 2565, 2566, 2567, 2132, 2568, 2569], 'similar_med': [2554, 2555, 1627, 2556, 1887, 2557, 2558, 2559, 2560, 2561]}
{'relevant': [2135], 'similar_high': [1383, 2136, 2137], 'similar_low': [1140, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148], 'similar_med': [2138, 2139]}
{'relevant': [3236], 'similar_high': [3237, 2109, 3238, 3239], 'similar_low': [3248, 3249, 3250, 3251, 3252, 3253, 3254, 3255, 3256, 3257], 'similar_med': [3240, 3241, 2850, 3242, 2839, 3243, 3244, 3245, 3246, 3247]}
{'relevant': [1979], 'similar_high': [727, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 953], 'similar_low': [1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005], 'similar_med': [957

In [80]:
doc_vectors = np.zeros((len(docs), EMBED_DIM), dtype=np.float64)

for i, doc in enumerate(docs):
    with open(doc) as fp:
        text = fp.read()
        text = re.sub('\n', ' ', text)
    try:
        doc_vectors[i] = embed_doc(doc, text)
    except:
        print('Error', doc, 'at', i, 'occurred.')

Error ../dataset/IR_dataset/2885.txt at 569 occurred.


In [86]:
sorted(list(zip([int(d.split('/')[-1].split('.')[0]) for d in docs], 
                np.dot(doc_vectors, q_vec) / (np.linalg.norm(q_vec) * np.linalg.norm(doc_vectors, axis=1)))), 
                key=lambda x: -x[1])[:20]

<ipython-input-86-1d7a7f331b3b>:2: RuntimeWarning: invalid value encountered in true_divide
  np.dot(doc_vectors, q_vec) / (np.linalg.norm(q_vec) * np.linalg.norm(doc_vectors, axis=1)))),


[(693, 0.7732056293722734),
 (2964, 0.6861036261664982),
 (2667, 0.6794550296090047),
 (225, 0.6743780133711479),
 (1971, 0.6712737159226223),
 (702, 0.6710633135617474),
 (496, 0.6536748979732013),
 (2965, 0.6532094453207982),
 (698, 0.6475874718421846),
 (1705, 0.647277432573165),
 (48, 0.6400741734479827),
 (3256, 0.6382923131028083),
 (2643, 0.6379608371356469),
 (3151, 0.6378615745589842),
 (2524, 0.6325519425081119),
 (1953, 0.6322054717449795),
 (104, 0.6315802982631057),
 (3236, 0.6286623733277574),
 (3055, 0.6284414570809459),
 (360, 0.6251452987920091)]

In [87]:
sorted(list(zip([int(d.split('/')[-1].split('.')[0]) for d in docs], np.dot(doc_vectors, q_vec))), 
       key=lambda x: -x[1])[:20]

[(100, 4.88990204693272),
 (693, 4.8091458808698775),
 (690, 4.774438527249401),
 (98, 4.43592442748941),
 (319, 4.356160218514877),
 (97, 4.347208878282771),
 (2667, 4.282882331320097),
 (689, 4.25378830226043),
 (320, 4.160616948221811),
 (2964, 4.154929720172538),
 (323, 4.152286463288714),
 (691, 4.148186248003334),
 (852, 4.143938147481116),
 (701, 4.139121811975143),
 (700, 4.122063291218833),
 (688, 4.102835803235924),
 (692, 4.102442926602915),
 (99, 4.094033596083273),
 (1631, 4.09306477797655),
 (3030, 4.088330743742448)]